## Градиентный спуск с постоянным шагом

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [31]:
from Functions.user_functions import f_well, f_poor, f_rozen, grad_well, grad_poor, grad_rozen
from Functions.optimization import classic_grad_descent

In [32]:
func_names = [f_well, f_poor, f_rozen]
grad_names = [grad_well, grad_poor, grad_rozen]
func_labels = ['Well-conditioned',
               'Poorly-conditioned',
               'Rosenbrock']

N_FUNC = len(func_names)

In [43]:
N_LR = 5
lr_classic_grad_descent = np.empty((N_FUNC, N_LR), dtype=object)

for ndx, lbl in enumerate(func_labels):
    
    match lbl:
        case 'Well-conditioned':
            for mdx in range(N_LR):
                lr_classic_grad_descent[ndx,mdx] = np.array([1/10**(mdx+1)] * 2)
        case 'Poorly-conditioned':
            lr_classic_grad_descent[ndx,0] = np.array([0.00010] * 2)
            lr_classic_grad_descent[ndx,1] = np.array([0.00008] * 2)
            lr_classic_grad_descent[ndx,2] = np.array([0.00006] * 2)
            lr_classic_grad_descent[ndx,3] = np.array([0.00004] * 2)
            lr_classic_grad_descent[ndx,4] = np.array([0.00002] * 2)
        case 'Rosenbrock':
            lr_classic_grad_descent[ndx,0] = np.array([0.00010] * 2)
            lr_classic_grad_descent[ndx,1] = np.array([0.00008] * 2)
            lr_classic_grad_descent[ndx,2] = np.array([0.00006] * 2)
            lr_classic_grad_descent[ndx,3] = np.array([0.00004] * 2)
            lr_classic_grad_descent[ndx,4] = np.array([0.00002] * 2)

for ndx, lbl in enumerate(func_labels):
    print(lbl)
    for lr in lr_classic_grad_descent[ndx,:]:
        print(lr)

Well-conditioned
[0.1 0.1]
[0.01 0.01]
[0.001 0.001]
[0.0001 0.0001]
[1.e-05 1.e-05]
Poorly-conditioned
[0.0001 0.0001]
[8.e-05 8.e-05]
[6.e-05 6.e-05]
[4.e-05 4.e-05]
[2.e-05 2.e-05]
Rosenbrock
[0.0001 0.0001]
[8.e-05 8.e-05]
[6.e-05 6.e-05]
[4.e-05 4.e-05]
[2.e-05 2.e-05]


In [44]:
'''
lr_classic_grad_descent = np.delete(lr_classic_grad_descent, [0,1,4])
for lr in lr_classic_grad_descent:
    print(lr)
N_LR = len(lr_classic_grad_descent)
'''

'\nlr_classic_grad_descent = np.delete(lr_classic_grad_descent, [0,1,4])\nfor lr in lr_classic_grad_descent:\n    print(lr)\nN_LR = len(lr_classic_grad_descent)\n'

In [45]:
N_EPS = 5
EPS_INIT = -1
eps_var = np.logspace(EPS_INIT, EPS_INIT-N_EPS+1, N_EPS)
print(eps_var)

[1.e-01 1.e-02 1.e-03 1.e-04 1.e-05]


In [46]:
x0_optim_arr     = np.empty((N_FUNC, N_LR), dtype='object')
x1_optim_arr     = np.empty((N_FUNC, N_LR), dtype='object')
iter_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
func_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
grad_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')

# Начальное приближение.
x_init = np.array([2.2, -2.2])

for ndx, func, grad in zip(range(N_FUNC), func_names, grad_names):
    for mdx, lr in enumerate(lr_classic_grad_descent[ndx,:]):
        x0_optim_list     = []
        x1_optim_list     = []
        iter_counter_list = []
        func_counter_list = []
        grad_counter_list = []
    
        for idx, eps in enumerate(eps_var):
            x_optim, _, iter_final, func_counter, grad_counter = classic_grad_descent(
                                                                                        loss_func=func,
                                                                                        grad_func=grad,
                                                                                        x_init=x_init,
                                                                                        learning_rate=lr,
                                                                                        tolerance=eps,
                                                                                        printoutput=False
                                                                                        )
            x0_optim_list.append(x_optim[0])
            x1_optim_list.append(x_optim[1])
            iter_counter_list.append(iter_final)
            func_counter_list.append(func_counter)
            grad_counter_list.append(grad_counter)
                       
        x0_optim_arr[ndx, mdx] = x0_optim_list
        x1_optim_arr[ndx, mdx] = x1_optim_list
        iter_counter_arr[ndx, mdx] = iter_counter_list
        func_counter_arr[ndx, mdx] = func_counter_list
        grad_counter_arr[ndx, mdx] = grad_counter_list

Преобразуем в таблицы.

In [47]:
col_name_arr = np.empty((N_FUNC,), dtype='object')
# Special column names for x_optim.
col_name_x_optim_arr = np.empty((N_FUNC,), dtype='object')

for ndx in range(N_FUNC):
    col_name = []
    col_name_x_optim = []
    for mdx in range(N_LR):
        col_name.append('x1: lr=' + str(lr_classic_grad_descent[ndx,mdx][0]) + ', x2: lr=' + str(lr_classic_grad_descent[ndx,mdx][1]))
        col_name_x_optim.append('x1 (lr=' + str(lr_classic_grad_descent[ndx,mdx][0]) + ')')
        col_name_x_optim.append('x2 (lr=' + str(lr_classic_grad_descent[ndx,mdx][1]) + ')')
    col_name_arr[ndx] = col_name.copy()
    col_name_x_optim_arr[ndx] = col_name_x_optim.copy()
    
x_optim_tbl_set      = np.empty((N_FUNC,), dtype=object)
iter_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
func_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
grad_counter_tbl_set = np.empty((N_FUNC,), dtype=object)

for ndx in range(N_FUNC):
    x_optim_tbl      = eps_var.copy()
    iter_counter_tbl = eps_var.copy()
    func_counter_tbl = eps_var.copy()
    grad_counter_tbl = eps_var.copy()
    
    for idx, colx1, colx2, col_iter, col_func, col_grad in zip(
                                                                range(N_LR),
                                                                x0_optim_arr[ndx, :],
                                                                x1_optim_arr[ndx, :],
                                                                iter_counter_arr[ndx, :],
                                                                func_counter_arr[ndx, :],
                                                                grad_counter_arr[ndx, :]
                                                                ):
        x_optim_tbl      = np.vstack((x_optim_tbl, colx1, colx2))
        iter_counter_tbl = np.vstack((iter_counter_tbl, col_iter))
        func_counter_tbl = np.vstack((func_counter_tbl, col_func))
        grad_counter_tbl = np.vstack((grad_counter_tbl, col_grad))
    
    x_optim_tbl_set[ndx] = pd.DataFrame(x_optim_tbl.T, columns=['eps'] + col_name_x_optim_arr[ndx])

    iter_counter_tbl_set[ndx] = pd.DataFrame(iter_counter_tbl.T, columns=['eps'] + col_name_arr[ndx])
    iter_counter_tbl_set[ndx][col_name_arr[ndx]] = iter_counter_tbl_set[ndx][col_name_arr[ndx]].astype(int)

    func_counter_tbl_set[ndx] = pd.DataFrame(func_counter_tbl.T, columns=['eps'] + col_name_arr[ndx])
    func_counter_tbl_set[ndx][col_name_arr[ndx]] = func_counter_tbl_set[ndx][col_name_arr[ndx]].astype(int)
    
    grad_counter_tbl_set[ndx] = pd.DataFrame(grad_counter_tbl.T, columns=['eps'] + col_name_arr[ndx])
    grad_counter_tbl_set[ndx][col_name_arr[ndx]] = grad_counter_tbl_set[ndx][col_name_arr[ndx]].astype(int)

In [48]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(x_optim_tbl_set[ndx])

Well-conditioned


,eps,x1 (lr=0.1),x2 (lr=0.1),x1 (lr=0.01),x2 (lr=0.01),x1 (lr=0.001),x2 (lr=0.001),x1 (lr=0.0001),x2 (lr=0.0001),x1 (lr=1e-05),x2 (lr=1e-05)
0,0.10000,0.047218,-0.020137,0.062427,-0.027395,0.063655,-0.028015,0.063759,-0.028069,0.063774,-0.028077
1,0.01000,0.005425,-0.002256,0.006323,-0.002644,0.006416,-0.002684,0.006430,-0.002690,0.006431,-0.002691
2,0.00100,0.000532,-0.000221,0.000627,-0.000260,0.000643,-0.000267,0.000644,-0.000267,0.000644,-0.000267
3,0.00010,0.000052,-0.000022,0.000063,-0.000026,0.000064,-0.000027,0.000064,-0.000027,0.000064,-0.000027
4,0.00001,0.000005,-0.000002,0.000006,-0.000003,0.000006,-0.000003,0.000006,-0.000003,0.000006,-0.000003


Poorly-conditioned


,eps,x1 (lr=0.0001),x2 (lr=0.0001),x1 (lr=8e-05),x2 (lr=8e-05),x1 (lr=6e-05),x2 (lr=6e-05),x1 (lr=4e-05),x2 (lr=4e-05),x1 (lr=2e-05),x2 (lr=2e-05)
0,0.10000,-5.492864,-0.207137,-5.492860,-0.207137,-5.492857,-0.207137,-5.492857,-0.207137,-5.492853,-0.207137
1,0.01000,-5.516819,-0.207727,-5.516818,-0.207727,-5.516818,-0.207727,-5.516818,-0.207727,-5.516818,-0.207727
2,0.00100,-5.519214,-0.207786,-5.519214,-0.207786,-5.519214,-0.207786,-5.519214,-0.207786,-5.519214,-0.207786
3,0.00010,-5.519454,-0.207792,-5.519454,-0.207792,-5.519454,-0.207792,-5.519454,-0.207792,-5.519454,-0.207792
4,0.00001,-5.519478,-0.207792,-5.519478,-0.207792,-5.519478,-0.207792,-5.519478,-0.207792,-5.519478,-0.207792


Rosenbrock


,eps,x1 (lr=0.0001),x2 (lr=0.0001),x1 (lr=8e-05),x2 (lr=8e-05),x1 (lr=6e-05),x2 (lr=6e-05),x1 (lr=4e-05),x2 (lr=4e-05),x1 (lr=2e-05),x2 (lr=2e-05)
0,0.10000,0.897286,0.804686,0.897286,0.804685,0.897286,0.804685,0.897284,0.804682,0.897283,0.804679
1,0.01000,0.988919,0.977917,0.988919,0.977916,0.988919,0.977916,0.988919,0.977916,0.988919,0.977916
2,0.00100,0.998883,0.997763,0.998883,0.997763,0.998883,0.997763,0.998883,0.997763,0.998883,0.997763
3,0.00010,0.999888,0.999776,0.999888,0.999776,0.999888,0.999776,0.999888,0.999776,0.999888,0.999776
4,0.00001,0.999989,0.999978,0.999989,0.999978,0.999989,0.999978,0.999989,0.999978,0.999989,0.999978


In [49]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(iter_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=0.1, x2: lr=0.1","x1: lr=0.01, x2: lr=0.01","x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05"
0,0.10000,26,259,2593,25935,259350
1,0.01000,40,418,4197,41982,419832
2,0.00100,55,578,5801,58034,580365
3,0.00010,70,737,7405,74087,740900
4,0.00001,85,896,9009,90139,901434


Poorly-conditioned


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,15083,18854,25139,37710,75421
1,0.01000,21216,26520,35361,53043,106089
2,0.00100,27349,34187,45583,68377,136758
3,0.00010,33481,41853,55806,83711,167426
4,0.00001,39614,49519,66028,99044,198095


Rosenbrock


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,30937,38666,51548,77312,154604
1,0.01000,83133,103911,138541,207803,415587
2,0.00100,140191,175234,233639,350449,700881
3,0.00010,197786,247228,329632,494439,988862
4,0.00001,255436,319291,425715,638565,1277115


In [50]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(func_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=0.1, x2: lr=0.1","x1: lr=0.01, x2: lr=0.01","x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05"
0,0.10000,26,259,2593,25935,259350
1,0.01000,40,418,4197,41982,419832
2,0.00100,55,578,5801,58034,580365
3,0.00010,70,737,7405,74087,740900
4,0.00001,85,896,9009,90139,901434


Poorly-conditioned


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,15083,18854,25139,37710,75421
1,0.01000,21216,26520,35361,53043,106089
2,0.00100,27349,34187,45583,68377,136758
3,0.00010,33481,41853,55806,83711,167426
4,0.00001,39614,49519,66028,99044,198095


Rosenbrock


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,30937,38666,51548,77312,154604
1,0.01000,83133,103911,138541,207803,415587
2,0.00100,140191,175234,233639,350449,700881
3,0.00010,197786,247228,329632,494439,988862
4,0.00001,255436,319291,425715,638565,1277115


In [51]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(grad_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=0.1, x2: lr=0.1","x1: lr=0.01, x2: lr=0.01","x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05"
0,0.10000,26,259,2593,25935,259350
1,0.01000,40,418,4197,41982,419832
2,0.00100,55,578,5801,58034,580365
3,0.00010,70,737,7405,74087,740900
4,0.00001,85,896,9009,90139,901434


Poorly-conditioned


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,15083,18854,25139,37710,75421
1,0.01000,21216,26520,35361,53043,106089
2,0.00100,27349,34187,45583,68377,136758
3,0.00010,33481,41853,55806,83711,167426
4,0.00001,39614,49519,66028,99044,198095


Rosenbrock


,eps,"x1: lr=0.0001, x2: lr=0.0001","x1: lr=8e-05, x2: lr=8e-05","x1: lr=6e-05, x2: lr=6e-05","x1: lr=4e-05, x2: lr=4e-05","x1: lr=2e-05, x2: lr=2e-05"
0,0.10000,30937,38666,51548,77312,154604
1,0.01000,83133,103911,138541,207803,415587
2,0.00100,140191,175234,233639,350449,700881
3,0.00010,197786,247228,329632,494439,988862
4,0.00001,255436,319291,425715,638565,1277115


# Графики

In [ ]:
user_figsize = (12, 6)
user_fontsize = 16

In [ ]:
for plt_lst, plt_name, fig_name_idx in zip(plot_list, plot_name, fig_name):
    plt.figure(figsize=user_figsize)
    plt.plot(eps_var, plt_lst[method_names], marker='.', markersize=12)
    plt.title(plt_name + ' (' + func_comment + ' функция)')
    plt.grid()
    plt.xscale('log')
    plt.xlabel('$\epsilon$', fontsize=user_fontsize)
    plt.legend(method_names)
    plt.savefig('readme_img/' + fig_name_idx + '_' + func_type.__name__ + '.png', bbox_inches='tight')
    plt.show()

In [ ]:
plt.figure(figsize=user_figsize)
for mdx, opt in enumerate(search_opt):
    plt.plot(eps_var, x_optim_arr[mdx], marker='.', markersize=12, label=opt)
plt.title('Минимум многомодальной функции')
plt.grid()
plt.xscale('log')
plt.ylabel('$x_{optim}$', fontsize=user_fontsize)
plt.xlabel('$\epsilon$', fontsize=user_fontsize)
plt.legend(search_opt)
plt.savefig('readme_img/x_optim_multimode.png', bbox_inches='tight')
plt.show()

# Генератор таблиц в приложения

In [ ]:
'''
def update_readme_section(tbl_df, readme_path, tbl_name, section):
    markdown_table = tbl_df.to_markdown(index=False)

    with open(readme_path, 'r', encoding="utf-8-sig") as f:
        content = f.read()

    # Define start and end markers.
    start_marker = f'<!-- START_{section.upper()} -->'
    end_marker = f'<!-- END_{section.upper()} -->'

    # Wrap the table with headers and markers.
    new_section = f'\n{start_marker} \n### {tbl_name}\n{markdown_table}\n{end_marker}'
    
    # Remove any previous content between the markers.
    pattern = re.compile(f'{re.escape(start_marker)}.*?{re.escape(end_marker)}', re.DOTALL)
    updated_content = pattern.sub("", content)

    # Insert the new section to the end of the file.
    updated_content += new_section

    with open(readme_path, 'w', encoding="utf-8-sig") as f:
        f.write(updated_content)
'''

In [ ]:
'''
section_names = ['x_optim', 'iter_counter', 'func_counter']

for plt_lst, tbl_tlt, section in zip(plot_list, tbl_title, section_names):
    update_readme_section(plt_lst, readme_path='const_step_descent.md', tbl_name=tbl_tlt, section=section + '_' + func_type.__name__)
'''